In [ ]:
import time
import numpy as np
import math
import numpy as np
from itertools import combinations
import os
import random
from collections import defaultdict

In [ ]:
def load_graph(file):
    '''Loads a graph from a .clq (DIMACS) file'''
    with open(os.path.join('data', file), "r") as f:
        lines = [line for line in f.readlines()]
    lines = [line.strip().split()
             for line in lines if line.strip().split() != []]
    p_line = [line for line in lines if line[0] == 'p'][0]
    e_lines = [line for line in lines if line[0] == 'e']
    n = int(p_line[2])
    print("Number of vertices:", n)
    print("Number of edges:", len(e_lines))
    adjmat = [[0] * n for _ in range(n)]
    for e in e_lines:
        v, w = int(e[1])-1, int(e[2])-1
        if v == w:
            print("Loop detected"), v
        if adjmat[v][w]:
            print("Duplicate edge"), v, w
        adjmat[v][w] = adjmat[v][w] = 1
    return (np.array(adjmat), n, len(e_lines))


def enter_matrix():
    R = int(input("Enter the number of rows:"))
    C = int(input("Enter the number of columns:"))
    print("Enter the entries in a single line (separated by space): ")
    # User input of entries in a
    # single line separated by space
    entries = list(map(int, input().split()))
    # For printing the matrix
    matrix = np.array(entries).reshape(R, C)
    print(repr(matrix))
    return(matrix)


def triangles(adj_mat):
    edges = all_edges(adj_mat)
    triangles = []
    for edge in edges:
        for neighbor in neighbors(edge[0], adj_mat):
            if is_edge(neighbor, edge[0], adj_mat) and is_edge(neighbor, edge[1], adj_mat):
                triangles.append(set([neighbor, edge[0], edge[1]]))
    return triangles


def neighbors(nodes, adj_mat):
    '''Takes one or a list of nodes (each one between 1 and N) and returns its neighbors (between 1 and N)'''
    neighbors = set()
    if isinstance(nodes, int):
        nodes = [nodes]
    for node in nodes:
        if node == 0:
            raise Exception("Node can't be 0: nodes are between 1 and N")

        for i in range(adj_mat.shape[0]):
            neighbor = adj_mat[node-1, i]
            if neighbor and (i != node):
                neighbors.add(i+1)
    return neighbors


def is_clique(nodes, adj_mat):
    for (node_i, node_j) in combinations(nodes, 2):
        if not is_edge(node_i, node_j, adj_mat):
            return False
    return True


def find_clique_dumb(node, adj_mat):
    '''Returns one clique to which the vertice "node" belongs'''
    clique = set([node])
    node_neighbors = set(neighbors(node, adj_mat))
    while node_neighbors:
        neighbor = node_neighbors.pop()
        if is_clique(clique.union(set([neighbor])), adj_mat):
            clique.add(neighbor)
    return clique


def is_edge(u, v, adj_mat):
    return adj_mat[u-1, v-1] or adj_mat[v-1, u-1]


def all_edges(adj_mat):
    n = adj_mat.shape[0]
    edges = set()
    for i in range(n):
        for j in range(i+1):
            if adj_mat[i, j]:
                edges.add((i+1, j+1))
    return edges


def is_in_clique(v, clique, adj_mat):
    is_in = True
    if clique is None:
        return False
    for node in clique:
        if not is_edge(v, node, adj_mat):
            is_in = False
    return is_in


def is_solution(nodes_list, adj_mat, v=None):
    "Verifies if the cliques in x up to v are indeed cliques"
    if v is None:
        v = adj_mat.shape[0]
    cliques_dict = cliques_from_list(nodes_list, v)
    for clique_nodes in cliques_dict.values():
        if not is_clique(clique_nodes, adj_mat):
            return False
    return True


def cliques_from_list(nodes_list, v=None):
    '''Takes the list X = [1 1 2 3 3 ... ] of nodes containing the label of their associated clique, and returns a dict of the different cliques'''
    if v is None:
        v = len(nodes_list)
    cliques = dict()
    for i in range(v):
        clique = nodes_list[i]
        if clique == 0:
            continue
        if clique in list(cliques):
            cliques[clique].add(i+1)
        else:
            cliques[clique] = set([i+1])
    return cliques

In [ ]:
def light_backtrack(adj_mat, cliques, v=1,  best=(math.inf, None)):

    n = adj_mat.shape[0]

    if v == n:
        if is_solution(cliques, adj_mat):
            if len(set(list(cliques))-set({0})) < best[0]:
                best = (len(set(list(cliques))), cliques_from_list(cliques))

    else:
        for i in range(1, v+2):
            cliques[v] = i
            if is_solution(cliques, adj_mat):
                if len(set(list(cliques))-set({0})) < best[0]:
                    best = light_backtrack(adj_mat, cliques, v+1, best)

    return best


def greedy(adj_mat, repetitions=10):
    n = adj_mat.shape[0]
    best = [x for x in range(1, n+1)]
    for r in range(repetitions):
        # print(best)
        vertices = [x for x in range(1, n+1)]
        random.shuffle(vertices)  # random permutation of vertices
        cliques = [0 for x in range(n)]
        sizes = defaultdict(int)  # size of each clique
        sizes[0] = n
        c = 1  # clique names
        for i in range(n):
            v = vertices[i]
            # print(" * Vertice", v)
            labeled = False
            # will count the size of each clique among the neighbors of v:
            neighbors_cliques = defaultdict(int)
            for neighbor in neighbors(v, adj_mat):
                neighbors_cliques[cliques[neighbor-1]] += 1
            for clique, size in neighbors_cliques.items():
                if size == sizes[clique]:
                    
                    cliques[v-1] = clique
                    sizes[clique] += 1
                    labeled = True
                    continue
            if not labeled:
                
                cliques[v-1] = c
                sizes[c] += 1
                c += 1
        if len(set(cliques)) < len(set(best)):
            best = cliques
        # print("Neighbors cliques", neighbors_cliques.keys())
    return best




def main(basePath,file):
    test_graph,_,_ = load_graph(basePath+file)

    start_time = time.time()
    solution = cliques_from_list(greedy(test_graph))
    print("File Name :",file)
    print("--- %s seconds ---" % (time.time() - start_time))
    print(len(solution), "cliques:", solution)

In [ ]:
#file = '/content/drive/MyDrive/Algorithm Dataset/Dataset only for pranto/brock200_2_b.txt'
#if __name__ == "__main__":
 #   main(file)
basePath = '/content/drive/MyDrive/Algorithm Dataset/Dataset only for pranto/'
files = [] 
for i in os.listdir(basePath): 
    if i.endswith('.txt'): 
        files.append(i) 
print(files)
print(len(files))

if __name__ == "__main__":
  for fileName in files:
    main(basePath,fileName)
    print()

['C125_9_clq.txt', 'C250_9_clq.txt', 'C500_9_clq.txt', 'C1000_9_clq.txt', 'brock200_2_b.txt', 'brock200_4_b.txt', 'brock400_2_b.txt', 'brock400_4_b.txt', 'brock800_2_b.txt', 'brock800_4_b.txt', 'gen200_p0_9_44_b.txt', 'gen200_p0_9_55_b.txt', 'gen400_p0_9_55_b.txt', 'gen400_p0_9_65_b.txt', 'gen400_p0_9_75_b.txt', 'p_hat300_1_clq.txt', 'p_hat300-2_clq.txt', 'p_hat700-1_clq.txt', 'p_hat1500-1_clq.txt', 'p_hat700-2_clq.txt', 'hamming10-4_clq.txt']
21
Number of vertices: 125
Number of edges: 6963
File Name : C125_9_clq.txt
--- 0.06797313690185547 seconds ---
55 cliques: {2: {1, 90}, 13: {2}, 45: {3}, 17: {4}, 36: {65, 58, 5, 70, 26}, 35: {6}, 15: {7}, 5: {8, 78, 95}, 26: {9}, 21: {10, 21}, 4: {11}, 44: {12, 38}, 34: {13, 31}, 7: {24, 14}, 3: {88, 59, 15}, 11: {16}, 51: {17, 69, 53, 39}, 47: {18}, 10: {19}, 32: {20}, 30: {22}, 38: {23}, 14: {25}, 33: {27}, 6: {57, 75, 28, 111}, 19: {29}, 52: {66, 30}, 53: {32, 91}, 27: {33, 100}, 39: {34}, 18: {35, 37}, 46: {48, 36}, 28: {40}, 25: {71, 104, 